# 3. Properties of gas mixtures

## 3.1. Binary CO$_2$-CH$_4$ mixture

In [ ]:
import numpy as np
import xarray as xr

from dartsflash.libflash import FlashParams, EoSParams, EoS, InitialGuess
from dartsflash.libflash import Flash
from dartsflash.libflash import CubicEoS

from dartsflash.mixtures import Mixture
from dartsflash.pyflash import PyFlash


mix = Mixture(components=["CO2", "C1"])
f = PyFlash(mixture=mix)

f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR),
          initial_guesses=[0, 1],
         )
ceos = f.eos["CEOS"]

f.init_flash(stabilityflash=True, eos_order=["CEOS"], np_max=2)

## 3.1a. PT-diagram

In [ ]:
""" Define state specifications and compositions """
dz = 0.01
min_z = [0.9]
max_z = [1.]
compositions = {"CO2": np.arange(min_z[0], max_z[0]+dz*0.1, dz),
                "C1": 1.,
               }
state_spec = {"pressure": np.arange(10, 200, 1),
              "temperature": np.arange(273.15, 423.15, 1),
              }

""" Perform flash calculations """
flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True)

In [ ]:
# Plot flash results
from dartsflash.plot import PlotFlash
pt = PlotFlash.pt(f, flash_results, composition=[0.9, 0.1], plot_phase_fractions=True)

## 3.1b. P-x and T-x diagrams

In [ ]:
""" Define state specifications and compositions """
dz = 0.01
min_z = [0.]
max_z = [1.]
compositions = {"CO2": np.arange(min_z[0], max_z[0]+dz*0.1, dz),
                "C1": 1.,
               }
state_spec = {
    "pressure": np.arange(1, 100, 1), "temperature": 273.15,  # uncomment for Px diagram
    # "temperature": np.arange(273.15, 573.15, 1), "pressure": 100.,  # uncomment for Tx diagram
}

flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True)

In [ ]:
# Plot flash results
from dartsflash.plot import PlotFlash
px = PlotFlash.binary(f, flash_results, variable_comp_idx=0, dz=dz, state=state_spec, min_z=min_z, max_z=max_z, plot_phase_fractions=True)

## 3.1c. PH diagram of binary mixture

In [ ]:
from dartsflash.libflash import PHFlash

components = ["CO2", "C1"]
mix = Mixture(components=components, setprops=True)

f = PyFlash(mixture=mix)

f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR),
          initial_guesses=[0, 1],
          switch_tol=1e-3)
f.flash_params.phflash_Htol = 1e-3
f.flash_params.phflash_Ttol = 1e-8

f.f = PHFlash(f.flash_params)
f.np_max = 2

In [ ]:
# Define range of P and T/H
z = [0.9, 0.1]
Trange = [250., 350.]
Prange = [10., 100.]
Hrange = [f.eos["CEOS"].H_PT(Prange[1], Ti, z) * 8.314472 for Ti in Trange]

state_spec = {"pressure": np.linspace(Prange[0], Prange[1], 200),
              "enthalpy": np.linspace(Hrange[0], Hrange[1], 100),
              }
compositions = {comp: z[i] for i, comp in enumerate(components)}

flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True)

from dartsflash.plot import PlotFlash
PlotFlash.ph(f, flash_results, composition=z, plot_temperature=True)

## 3.2. Ternary mixture with CO$_2$

In [ ]:
import numpy as np
import xarray as xr

from dartsflash.libflash import FlashParams, EoSParams, EoS, InitialGuess
from dartsflash.libflash import Flash
from dartsflash.libflash import CubicEoS

from dartsflash.mixtures import Mixture
from dartsflash.pyflash import PyFlash

# Redefine mixture
components = ["CO2", "C1", "H2S"]
mix = Mixture(components)
f = PyFlash(mixture=mix)

f.add_eos("CEOS", CubicEoS(mix.comp_data, CubicEoS.PR),
          initial_guesses=[InitialGuess.Wilson, 0, 2], switch_tol=1e-2)
ceos = f.eos["CEOS"]

f.flash_params.stability_variables = FlashParams.alpha
f.flash_params.split_variables = FlashParams.nik
f.flash_params.split_tol = 1e-20
f.flash_params.split_switch_tol = 1e-2
f.flash_params.tpd_tol = 1e-11
f.flash_params.tpd_close_to_boundary = 1e-2

f.init_flash(stabilityflash=True, eos_order=["CEOS"], np_max=3)

## 3.2a. PT-diagram

In [ ]:
""" Define state specifications and compositions """
state_spec = {"pressure": np.arange(10, 200, 1),
              "temperature": np.arange(273.15, 423.15, 1),
              }
composition = [0.6, 0.2, 0.2]
compositions = {components[i]: composition[i] for i in range(3)}

""" Perform flash calculations """
flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True)

In [ ]:
# Plot flash results
from dartsflash.plot import PlotFlash
pt = PlotFlash.pt(f, flash_results, composition=composition, plot_phase_fractions=True)

## 3.2b. Ternary diagram

In [ ]:
""" Define state specifications and compositions """
state_spec = {"pressure": np.array([60.]),
              "temperature": np.array([273.15]),
              }
dz = 0.005
min_z = [0., 0.]
max_z = [1., 1.]
compositions = {comp: np.arange(min_z[i], max_z[i]+0.1*dz, dz) for i, comp in enumerate(components[:-1])}
compositions[components[-1]] = 1.

""" Perform flash calculations """
flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True)

In [ ]:
""" Plot flash results """
PlotFlash.ternary(f, flash_results, dz=dz, min_z=min_z, max_z=max_z, state={spec: state_spec[spec] for spec in state_spec.keys()}, composition_to_plot=composition)